In [7]:
%%time


import pandas as pd
import numpy as np
import re
import nltk

from nltk.tokenize import word_tokenize

from sentence_transformers import SentenceTransformer, util
from concurrent import futures
from tqdm import tqdm
import pymorphy2

# model.py
# Функция для добавления пропущенных пробелов в наименованиях.
def add_spaces(text):
    spaced_text = re.sub(r'(?<=[a-zA-Z])(?=[а-яА-ЯёЁ])|(?<=[а-яА-ЯёЁ])(?=[a-zA-Z])', ' ', text)
    spaced_text = re.sub(r'(\S)\*(\S)', r'\1 * \2', spaced_text)
    spaced_text = re.sub(r'(\d+)([а-яА-ЯёЁa-zA-Z]+)', r'\1 \2', spaced_text)
    return spaced_text


# Функция для извлечения концентрации, объема, единицы измерения и фасовки из наименования товара в отдельные столбцы.
def extract_info(data):
    pattern_volume = r'\s*(\d+(?:[.,]\d+)?)\s*([мк]?[лг]|лит[р]?[ы]?|к[г]?[р]?[ам]?[ы]?)\s*'
    pattern_concentration = r'\s*(\d+:\d+)\s*'
    pattern_quantity = r'\s*(\d+)\s*(?:шт(?:ук[и]?|\.)?|ШТ|шт)\s*'

    data['volume'] = data['name'].str.extract(pattern_volume, expand=True)[0]
    data['units_of_meas'] = data['name'].str.extract(pattern_volume, expand=True)[1]
    data['concent'] = data['name'].str.extract(pattern_concentration, expand=False)
    data['units'] = data['name'].str.extract(pattern_quantity, expand=False)
    
    data['units'].fillna('1', inplace=True)
    data['concent'].fillna('1:1', inplace=True)
    data['volume'].fillna('1', inplace=True)
    data['units_of_meas'].fillna('шт', inplace=True)
    
    # Добавляем ваш фрагмент кода для удаления лишних пробелов между буквой и словом
    data['name'] = data['name'].str.replace(r'(\w)\s+(?=\w)', r'\1')

    return data


# Функция для извлечения первых 3-4 слов из названия товара
def extract_keywords(name):
    words = name.split()[:4]
    return ' '.join(words)


# Функция очистки,  токенизации, лемматизации и удаление стоп-слов.
def preprocess_text(text):
    # Очистка текста
    cleaned_text = re.sub(r"[\(\),/]", ' ', text) # r"[^a-zA-Zа-яА-ЯёЁ ]"
    
    # Токенизация
    tokens = word_tokenize(cleaned_text.lower())
    
    # Лемматизация
#     lemmatizer = WordNetLemmatizer()
#     lemmas = [lemmatizer.lemmatize(token) for token in tokens]
    
    # Лемматизация ттолько русских слов, английские являются неизменяемыми названиями
    morph = pymorphy2.MorphAnalyzer()
    lemmas = [morph.parse(word)[0][2] for word in tokens]
    
    # Удаление стоп-слов
#     stop_words = set(stopwords.words('russian') + stopwords.words('english'))
#     filtered_words = [lemma for lemma in lemmas if lemma not in stop_words]
    
    # Возвращение предобработанного текста
    return ' '.join(lemmas)

# функция для использования функций обработки текста / предсказания по батчам с параллельными вычислениями
def use_batch_get_rec(data, func_get_rec, batch_size = 2000):
  
    # Создание пула потоков # параллельные вычисления
    executor = futures.ThreadPoolExecutor()
    
    recommendations = []

    # Разделение данных на пакеты и обработка пакетов
    for i in range(0, len(data), batch_size):
        batch = data['product_name'].iloc[i:i+batch_size]
#         batch_recommendations = batch.progress_apply(lambda x: func_get_rec(x))
        batch_recommendations = list(tqdm(executor.map(func_get_rec, batch), total=len(batch)))
        recommendations.extend(batch_recommendations)
        
    return recommendations

def get_rec_labse(product_name):
    preprocessed_name = preprocess_text(product_name)

    dealer_product_embedding = model_labse.encode([preprocessed_name])

    # Поиск наиболее похожих названий
    cosine_scores = util.pytorch_cos_sim(dealer_product_embedding, corpus_embeddings_labse)[0]

    top_matches_indices = cosine_scores.argsort(descending=True)[:5]
#     top_matches_names = [names_corpus[i] for i in top_matches_indices]
    recommendations = rec_article = data_products.loc[list(top_matches_indices)]['name'].tolist()

    return recommendations


def matching(product_name):
    
    data_products = pd.read_csv('marketing_product.csv')


    # удалили пропуск в названии 2 шт
    data_products.dropna(subset=['name'], inplace=True)
    # удалили одно название пустое
    empty_name_rows = data_products[data_products['name'].str.contains(r'^\s*$')]
    data_products.drop(index=empty_name_rows.index, inplace=True)


    #  добавить пропущенные пробелы в наименования товаров в столбцах 'name', 'name_1c', 'ozon_name' и 'wb_name'
    columns_to_apply = ['name', 'name_1c', 'ozon_name', 'wb_name']
    data_products[columns_to_apply] = data_products[columns_to_apply].astype(str).applymap(add_spaces)

    #  обьем, количество штук в упаковке, концентрация продукция, разная единица измерения
    # извлечь и зафиксировать в отдельные столбцы в качестве дополнительных признаков
    data_products = extract_info(data_products)

    # обнаружено, что есть лишний пробел в слове Средство "C редство"
    # вручную исправим ошибку
    data_products['ozon_name'] = data_products['ozon_name'].str.replace('C редство', 'Средство')
    data_products['name_1c'] = data_products['name_1c'].str.replace('C редство', 'Средство')
    data_products['wb_name'] = data_products['wb_name'].str.replace('C редство', 'Средство')
    data_products['name'] = data_products['name'].str.replace('C редство', 'Средство')

    # в наименования оказались еще места, которые не были обработаны функцией и уберем эти проблемы вручную
    data_products['name'] = data_products['name'].str.replace('БЕТОНКОНТАКТготовый', 'БЕТОНКОНТАКТ готовый')
    data_products['name'] = data_products['name'].str.replace('"к', '" к')
    data_products['name'] = data_products['name'].str.replace('яблокаконцентрированное', 'яблока концентрированное')
    data_products['name'] = data_products['name'].str.replace('(сухой остаток 20%)', ' (сухой остаток 20%) ')
    data_products['name'] = data_products['name'].str.replace('.C', '. C')

    # пропуски в столбце name_1c заполнить значениями в сттолбце name 
    data_products['name_1c'].fillna(data_products['name'], inplace=True)

    # очистка названий товаров заказчика в data_products
    data_products['name_clean'] = data_products['name_1c'].progress_apply(lambda x: preprocess_text(x))

    data_dealers['product_name_clean'] = use_batch_get_rec(data_dealers, preprocess_text)


    # # Заполнение пропущенных значений в категориях на основе группировки по первым 3-4 словам в названиях товаров
    # data_products['category_id'].fillna(data_products['name'].apply(extract_keywords), inplace=True)

    # # Для пропусков в ценах поставим просто заглушку -1
    # data_products['cost'].fillna('-1', inplace=True)
    # data_products['recommended_price'].fillna('-1', inplace=True)





    #  оставляем необходимый набор признаков 
    # соединим ключи по id с названиями производителя
    product_with_key_data = data_key[['key', 'product_id']].merge(
        data_products[['id', 'name','name_clean','cost','recommended_price','category_id',
                        'volume', 'units_of_meas', 'concent', 'units']],  
        how='left', 
        right_on='id', 
        left_on='product_id'
    )
    # к датасету с названиями дилеров присоединим наззвания производителя 
    full_data = data_dealers[ ['product_key', 'product_name','product_name_clean', 'price']].merge( 
        product_with_key_data, 
        how='left', 
        right_on='key', 
        left_on='product_key').drop(['product_id', 'product_key'], axis=1)
    full_data.sample(2)

    #  удалим неразмеченные записи и перезапишем индексы
    full_data = full_data.dropna().reset_index(drop=True)
    data_products = data_products.dropna( subset=['name']).reset_index(drop=True)


    # заготовим корпуса названий производителя
    names_corpus = data_products['name_1c'].tolist()


    model_labse = SentenceTransformer('sentence-transformers/LaBSE')

    corpus_embeddings_labse = model_labse.encode(names_corpus)
    
    return get_rec_labse(product_name)

product_name = 'Средство моющее для бани и сауны Prosept Multipower Wood 1л'


matching(product_name)

ParserError: Error tokenizing data. C error: Expected 7 fields in line 4, saw 9
